# Importing libraries

In [1]:
import os, glob, json, csv, subprocess, sys, re, operator, itertools
import datetime
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Iterable
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository and directories

In [2]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/hbase/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Loading comparison data between bugline and blame

In [3]:
compare_dirs = analyze_dir + '04_annotate/02_diff-file_blame-file_comparison/'

In [4]:
dirs = []
for d in glob.iglob(compare_dirs + '*', recursive=True):
    dirs.append(d)
dirs.sort()
dirs

['/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/02_diff-file_blame-file_comparison/histogram_comparison',
 '/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/02_diff-file_blame-file_comparison/minimal_comparison',
 '/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/02_diff-file_blame-file_comparison/myers_comparison',
 '/Users/YusufNugroho/different-diff/analyze/analyze_hbase/04_annotate/02_diff-file_blame-file_comparison/patience_comparison']

In [5]:
column_name = ['bug_id','line_number','buggy_code','does_bugline_from_originfile?','date_added_in_originfile','earliest_affected_version','version_release_date','bug_class']

# Reading files from comparison dataset

In [6]:
#Reading files from comparison
for n, folder in enumerate(dirs):
    result = []
    z = 1
    for file in glob.iglob(folder + '/*', recursive=True):
        sys.stdout.write("\r{} Reading file no: {}".format((n+1),z))
        sys.stdout.flush()
        z += 1
        res = []
        with open(file,'r') as myfile:
            csv_reader = csv.reader(myfile, delimiter=',')
            for row in csv_reader:
                res.append(row[7])
        del res[0]

        regex = re.search("\/diff_(?P<filename>\w.*)\_(?P<algorithm>\w+)\-\w+\_\w.*(?P<commitid>[a-z0-9]{10})-(?P<parentid>[a-z0-9]{10})\_HBASE-[0-9]+\_(?P<filenumber>\d+)", file)
        regex = regex.groups()

        if ('bug-introducing change') not in res:
            tmp = [file,row[0],regex[0],regex[2],regex[3],int(regex[4]),"Does not affect the version"]
        else:
            tmp = [file,row[0],regex[0],regex[2],regex[3],int(regex[4]),"Affect the version"]
        result.append(tmp)
    result = sorted(result, key=operator.itemgetter(5))
    
    with open(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/" + regex[1] + "_affected_files.csv",'w') as csvfile:
        fields = ['file','bug_id','filename','bugintro_commitID','parent_id','filenumber','affect_version?']
        writers = csv.writer(csvfile, delimiter=",")
        writers.writerow(fields)
        for item in result:
            writers.writerow(item)
            
print ("\nThe process is complete")

4 Reading file no: 2562
The process is complete


In [7]:
df_affhist = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/histogram_affected_files.csv")
df_affhist = df_affhist[fields]
df_affhist

,file,bug_id,filename,bugintro_commitID,parent_id,filenumber,affect_version?
0,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-17624,AbstractFSWALProvider.java,e019961150,b2217d1d8f,1,Does not affect the version
1,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-11258,AbstractHBaseTool.java,45883c6bd1,2ab2e5c40e,2,Does not affect the version
2,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-16620,AbstractHBaseTool.java,e5d77d095c,44812cf1ed,3,Does not affect the version
3,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15650,AbstractMemStore.java,d815211f5a,6930da781b,4,Affect the version
4,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-3100,AbstractMergeTestBase.java,af7f5c7d01,516667bc74,5,Does not affect the version
5,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-19319,AbstractProcedureScheduler.java,f886716617,5a0881a98b,6,Does not affect the version
6,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractProtobufLogWriter.java,95c1dc93fb,cc237c4b78,7,Does not affect the version
7,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRegionServerCallable.java,95c1dc93fb,cc237c4b78,8,Does not affect the version
8,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRpcClient.java,95c1dc93fb,cc237c4b78,9,Does not affect the version
9,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-18946,AbstractTestDLS.java,010012cbcb,11e82de8a1,10,Affect the version


In [8]:
df_affmin = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/minimal_affected_files.csv")
df_affmin = df_affmin[fields]
df_affmin

,file,bug_id,filename,bugintro_commitID,parent_id,filenumber,affect_version?
0,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-17624,AbstractFSWALProvider.java,e019961150,b2217d1d8f,1,Does not affect the version
1,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-11258,AbstractHBaseTool.java,45883c6bd1,2ab2e5c40e,2,Does not affect the version
2,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-16620,AbstractHBaseTool.java,e5d77d095c,44812cf1ed,3,Does not affect the version
3,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15650,AbstractMemStore.java,d815211f5a,6930da781b,4,Affect the version
4,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-3100,AbstractMergeTestBase.java,af7f5c7d01,516667bc74,5,Does not affect the version
5,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-19319,AbstractProcedureScheduler.java,f886716617,5a0881a98b,6,Does not affect the version
6,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractProtobufLogWriter.java,95c1dc93fb,cc237c4b78,7,Does not affect the version
7,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRegionServerCallable.java,95c1dc93fb,cc237c4b78,8,Does not affect the version
8,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRpcClient.java,95c1dc93fb,cc237c4b78,9,Does not affect the version
9,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-18946,AbstractTestDLS.java,010012cbcb,11e82de8a1,10,Affect the version


In [9]:
df_affmyers = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/myers_affected_files.csv")
df_affmyers = df_affmyers[fields]
df_affmyers

,file,bug_id,filename,bugintro_commitID,parent_id,filenumber,affect_version?
0,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-17624,AbstractFSWALProvider.java,e019961150,b2217d1d8f,1,Does not affect the version
1,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-11258,AbstractHBaseTool.java,45883c6bd1,2ab2e5c40e,2,Does not affect the version
2,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-16620,AbstractHBaseTool.java,e5d77d095c,44812cf1ed,3,Does not affect the version
3,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15650,AbstractMemStore.java,d815211f5a,6930da781b,4,Affect the version
4,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-3100,AbstractMergeTestBase.java,af7f5c7d01,516667bc74,5,Does not affect the version
5,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-19319,AbstractProcedureScheduler.java,f886716617,5a0881a98b,6,Does not affect the version
6,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractProtobufLogWriter.java,95c1dc93fb,cc237c4b78,7,Does not affect the version
7,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRegionServerCallable.java,95c1dc93fb,cc237c4b78,8,Does not affect the version
8,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRpcClient.java,95c1dc93fb,cc237c4b78,9,Does not affect the version
9,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-18946,AbstractTestDLS.java,010012cbcb,11e82de8a1,10,Affect the version


In [10]:
df_affpat = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/patience_affected_files.csv")
df_affpat = df_affpat[fields]
df_affpat

,file,bug_id,filename,bugintro_commitID,parent_id,filenumber,affect_version?
0,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-17624,AbstractFSWALProvider.java,e019961150,b2217d1d8f,1,Does not affect the version
1,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-11258,AbstractHBaseTool.java,45883c6bd1,2ab2e5c40e,2,Does not affect the version
2,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-16620,AbstractHBaseTool.java,e5d77d095c,44812cf1ed,3,Does not affect the version
3,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15650,AbstractMemStore.java,d815211f5a,6930da781b,4,Affect the version
4,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-3100,AbstractMergeTestBase.java,af7f5c7d01,516667bc74,5,Does not affect the version
5,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-19319,AbstractProcedureScheduler.java,f886716617,5a0881a98b,6,Does not affect the version
6,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractProtobufLogWriter.java,95c1dc93fb,cc237c4b78,7,Does not affect the version
7,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRegionServerCallable.java,95c1dc93fb,cc237c4b78,8,Does not affect the version
8,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-15638,AbstractRpcClient.java,95c1dc93fb,cc237c4b78,9,Does not affect the version
9,/Users/YusufNugroho/different-diff/analyze/ana...,HBASE-18946,AbstractTestDLS.java,010012cbcb,11e82de8a1,10,Affect the version


# Counting affected and non-affected files

In [11]:
df = [df_affhist,df_affmin,df_affmyers,df_affmyers]

In [12]:
algorithms = ['histogram','minimal','myers','patience']

In [13]:
no_of_files = []
for z, dframe in enumerate(df):
    affnum = dframe['affect_version?'][dframe['affect_version?'] == 'Does not affect the version'].count()
    nonaffnum = dframe['affect_version?'][dframe['affect_version?'] == 'Affect the version'].count()
    summ = [algorithms[z],affnum,nonaffnum]
    no_of_files.append(summ)

In [14]:
with open(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/totalnumber_of_affectedfiles.csv",'w') as file:
    header = ['algorithm','#non-affected_files','#affected_files']
    writers = csv.writer(file,delimiter=",")
    writers.writerow(header)
    for ii in no_of_files:
        writers.writerow(ii)
print ("File 'totalnumber_of_affectedfiles.csv' has been created")

File 'totalnumber_of_affectedfiles.csv' has been created


In [15]:
aff_file = pd.read_csv(analyze_dir + "04_annotate/04_grouping_affected_non-affected_files/totalnumber_of_affectedfiles.csv")
aff_file

,algorithm,#non-affected_files,#affected_files
0,histogram,1782,780
1,minimal,1782,780
2,myers,1782,780
3,patience,1782,780
